>### 🚩 *Create a free WhyLabs account to get more value out of whylogs!*<br> 
>*Did you know you can store, visualize, and monitor whylogs profiles with the [WhyLabs Observability Platform](https://whylabs.ai/whylogs-free-signup?utm_source=whylogs-Github&utm_medium=whylogs-example&utm_campaign=Condition_Validators)? Sign up for a [free WhyLabs account](https://whylabs.ai/whylogs-free-signup?utm_source=whylogs-Github&utm_medium=whylogs-example&utm_campaign=Condition_Validators) to leverage the power of whylogs and WhyLabs together!*

# Real-time Data Validation with Condition Validators

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/whylabs/whylogs/blob/mainline/python/examples/advanced/Condition_Validators.ipynb)

With __Condition Validators__, the user is able to evaluate conditions on individual values on real-time scenarios. These checks are done while data is being logged, and can trigger one or multiple actions when these conditions fail to be met. With __Condition Validators__, you are able to define actions where an immediate response is required, such as emiting an alert to key stakeholders, logging specific failures or throwing exceptions. Validators are designed with flexibility in mind, so you are free to customize your actions as well as the conditions that trigger those actions.

In this example, we will cover how to:

- Define conditions for data validations
- Define actions to be triggered when conditions fail
- Assemble a Condition Validator with defined conditions and actions
- Bind validators to Columns
- Apply validators to your data

Showing the different types of conditions is NOT the focus of this example. If you wish to see the different types of conditions you can define, please refer to [Condition Count Metrics](https://whylogs.readthedocs.io/en/stable/examples/advanced/Condition_Count_Metrics.html).

Unlike metrics, validators will not log properties into profiles. They are meant only to evaluate conditions and trigger actions while logging is under way.



## Use-case: Validating email and credit card columns

In this simple scenario, we want to make sure two things happen:

- Single emails in the `emails` column (nothing else)
- No credit cards information in the `transcriptions` column


We'll use the following sample dataframe to evaluate on:

In [1]:
import pandas as pd

text_data = {
    "emails": [
        "my email is my_email_1989@gmail.com",
        "invalidEmail@xyz.toolong",
        "this.is.ok@hotmail.com",
        "not an email",
    ],
    "transcriptions": [
        "Bob's credit card number is 4000000000000",
        "Alice's credit card is XXXXXXXXXXXXX",
        "Hi, my name is Bob",
        "Hi, I'm Alice",
    ],
}

df = pd.DataFrame(data=text_data)


## Defining the Conditions

Let's translate the mentioned conditions into regex expressions:
- A negation of a credit card pattern matching ("No Credit Cards")
- A Full Match for an email pattern matching:

> Our conditions are usually expected to evaluate to `True`. When something goes wrong, the condition should evaluate to `False`, triggering a certain action in the process. This is why we negate the first condition (because matching the pattern is bad) and do a match for the second one (because not finding an email is bad)

In [2]:
from whylogs.core.metrics.condition_count_metric import Condition
from whylogs.core.metrics.condition_count_metric import Relation as Rel
from whylogs.core.metrics.condition_count_metric import relation as rel
from whylogs.core.metrics.condition_count_metric import not_relation as not_rel


credit_card_conditions = {"noCreditCard": Condition(not_rel(rel(Rel.match, ".*4[0-9]{12}(?:[0-9]{3})?")))}

email_conditions = {"hasEmail": Condition(rel(Rel.fullmatch, "[\w.]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}"))}

> Note: The regex expressions are for demonstrational purposes only. These expressions are not general - there will be emails and credit cards whose patterns will not be met by the expression.

## Defining the actions

The action to be triggered when a contidion fails is created by simply defining a regular function.

We should just remember to define the arguments: `validator_name`, `condition_name` and `value`. You can use these values to help with logging and debugging the failures.

In [3]:
from typing import Any

def do_something_important(validator_name, condition_name: str, value: Any):
    print("Validator: {}\n    Condition name {} failed for value {}".format(validator_name, condition_name, value))
    return


## Creating the Validators

To create a Condition Validator, we need a name, a set of conditions, and a list of actions.

Let's make a Validator for the credit card column and another Validator for the email column. Each validator has a single condition to be evaluated, and also a single action.


> Note that for a single validator, we could have multiple conditions defined and also multiple actions to be triggered.

In [4]:
from whylogs.core.validators import ConditionValidator


credit_card_validator = ConditionValidator(
    name="no_credit_cards",
    conditions=credit_card_conditions,
    actions=[do_something_important],
)

email_validator = ConditionValidator(
    name="has_emails",
    conditions=email_conditions,
    actions=[do_something_important],
)


## Bind the Validators to specific Columns

Each validator instance should be mapped to a single column, but each column can have multiple validators attached to it.

Assigning an instance to multiple columns will lead to an undefined behavior.

In our case, we have only one validator for each of the columns:

In [5]:
validators = {
    "emails": [email_validator],
    "transcriptions": [credit_card_validator]}

## Apply Validators to Data

Now, we only need to pass our set of validators to our DatasetSchema.

This will make the validators to be applied while data is being logged. The actions will be triggered immediately when the conditions fail, and not only when the logging is done.

In [6]:
from whylogs.core.schema import DatasetSchema
import whylogs as why

schema = DatasetSchema(validators=validators)
profile = why.log(df, schema=schema).profile()

Validator: has_emails
    Condition name hasEmail failed for value my email is my_email_1989@gmail.com
Validator: has_emails
    Condition name hasEmail failed for value invalidEmail@xyz.toolong
Validator: has_emails
    Condition name hasEmail failed for value not an email
Validator: no_credit_cards
    Condition name noCreditCard failed for value Bob's credit card number is 4000000000000


We can see in the results above that our `has_emails` validator failed three times. The first time, the value has extra text, the second has an invalid email address and the third does not contain an email.

The `no_credit_cards` validator failed once, because the pattern was found once.

We can also access a simple summary with the total number of evaluations, the number of total failures and the number of failures per condition present in the validator:

In [7]:
email_validator.to_summary_dict()

{'total_evaluations': 4, 'hasEmail': 3}

In [8]:
credit_card_validator.to_summary_dict()

{'total_evaluations': 4, 'noCreditCard': 1}